# Processamento dados

## SETUP

### import

In [26]:
import os
import json
import pandas as pd

### Mount Drive

In [27]:
# drive.mount('/content/drive')
# from google.colab import drive

### Constants

In [28]:
# BRONZE_PATH = '/content/drive/MyDrive/Tech Challenge 4/Obesity.csv'
# SILVER_PATH = '/content/drive/MyDrive/Tech Challenge 4/Rafa'

BASE_PATH = '/home/lucas-nunes/workspace/Postech/challenges/4_obesity/data/'
BRONZE_PATH = os.path.join(BASE_PATH, 'bronze')
SILVER_PATH = os.path.join(BASE_PATH, 'silver')

FILENAME = 'Obesity.csv'

INPUT_FILE_PATH = os.path.join(BRONZE_PATH, FILENAME)

if not os.path.exists(SILVER_PATH): os.makedirs(SILVER_PATH, exist_ok=True)

### VALUE DICTS

In [ ]:
DICT_HEADER_OBESITY_TRANSLATION = {
  'gender': "Gênero",
  'age': "Idade",
  'height': "Altura em metros",
  'weight': "Peso em kgs",
  'family_history': "Algum membro da família sofreu ou sofre de excesso de peso?",
  'favc': "Você come alimentos altamente calóricos com frequência?",
  'fcvc': "Você costuma comer vegetais nas suas refeições?",
  'ncp': "Quantas refeições principais você faz diariamente?",
  'caec': "Você come alguma coisa entre as refeições?",
  'smoke': "Você fuma?",
  'ch2o': "Quanta água você bebe diariamente?",
  'scc': "Você monitora as calorias que ingere diariamente?",
  'faf': "Com que frequência você pratica atividade física?",
  'tue': "Quanto tempo você usa dispositivos tecnológicos como celular, videogame, televisão, computador e outros?",
  'calc': "Com que frequência você bebe álcool?",
  'mtrans': "Qual meio de transporte você costuma usar?",
  'obesity': "Nível de obesidade"
}

DICT_VALUE_OBESITY_TRANSLATION = {
  "Female": "Feminino",
  "Male": "Masculino",
  "yes": "Sim",
  "no": "Não",
  "Sometimes": "Às vezes",
  "Frequently": "Frequentemente",
  "Always": "Sempre",
  "Public_Transportation": "Transporte público",
  "Walking": "Caminhada",
  "Automobile": "Automóvel",
  "Motorbike": "Motocicleta",
  "Bike": "Bicicleta",
  "Normal_Weight": "Peso normal",
  "Overweight_Level_I": "Sobrepeso Nível I",
  "Overweight_Level_II": "Sobrepeso Nível II",
  "Obesity_Type_I": "Obesidade Tipo I",
  "Obesity_Type_II": "Obesidade Tipo II",
  "Obesity_Type_III": "Obesidade Tipo III",
  "Insufficient_Weight": "Peso insuficiente"
}
# a
DICT_FREQUENCY_OBESITY = {
  'no': 0,
  'sometimes': 1,
  'frequently': 2,
  'always': 3
}

DICT_LEVEL_OBESITY = {
  'insufficient_weight': 0,
  'normal_weight': 1,
  'overweight_level_i': 2,
  'overweight_level_ii': 3,
  'obesity_type_i': 4,
  'obesity_type_ii': 5,
  'obesity_type_iii': 6
}

DICT_YES_NO_TO_BOOL = {
  'yes': True,
  'no': False
}

## Read

In [30]:
df = pd.read_csv(INPUT_FILE_PATH, dtype=str)

## Process

### Processing functions

In [31]:
def convert_to_int(value):
    value = int(round(float(value), 0))
    return value


def convert_to_bool(value):
    if value == 'yes':
        return True
    elif value == 'no':
        return False
    else:
        raise KeyError(f'item: {value} is not "yes" or "no"')

    
def convert_dict_to_int(value, dict_reference):
    value = str(value).lower()
    if value in dict_reference:
        return dict_reference[value]
    else:
        raise KeyError(f'value: {value} is not one of the keys: {list(dict_reference.keys())}')


def rename_column_to_lower(df):
    dict_values = {value:str(value).strip().lower() for value in df.columns.values}
    df.rename(columns=dict_values, inplace=True)
    return df


def process_all_values(df):
    df['gender'] = df['gender'].astype(str)
    df['age'] = df['age'].T.apply(convert_to_int)
    df['height'] = df['height'].astype(float)
    df['weight'] = df['weight'].astype(float)
    df['family_history'] = df['family_history'].T.apply(convert_dict_to_int, dict_reference=DICT_YES_NO_TO_BOOL)
    df['favc'] = df['favc'].T.apply(convert_dict_to_int, dict_reference=DICT_YES_NO_TO_BOOL)
    df['fcvc'] = df['fcvc'].astype(float)
    df['ncp'] = df['ncp'].T.apply(float)
    df['caec'] = df['caec'].T.apply(convert_dict_to_int, dict_reference=DICT_FREQUENCY_OBESITY)
    df['smoke'] = df['smoke'].T.apply(convert_dict_to_int, dict_reference=DICT_YES_NO_TO_BOOL)
    df['ch2o'] = df['ch2o'].T.apply(float)
    df['scc'] = df['scc'].T.apply(convert_dict_to_int, dict_reference=DICT_YES_NO_TO_BOOL)
    df['faf'] = df['faf'].T.apply(float)
    df['tue'] = df['tue'].T.apply(float)
    df['calc'] = df['calc'].T.apply(convert_dict_to_int, dict_reference=DICT_FREQUENCY_OBESITY)
    df['mtrans'] = df['mtrans'].astype(str)
    df['obesity'] = df['obesity'].T.apply(convert_dict_to_int, dict_reference=DICT_LEVEL_OBESITY)
    return df

def translate_value_fields(df):
    df.replace(DICT_VALUE_OBESITY_TRANSLATION, inplace=True)
    return df

### Process Execution

In [32]:
df = rename_column_to_lower(df)
df = process_all_values(df)
df = translate_value_fields(df)

## Write

### Write Functions

In [33]:
def write_json_file(filename: str, content : dict, base_path: str = SILVER_PATH, encoding: str = 'utf-8'):
    full_path = os.path.join(base_path, f'{filename}.json')

    with open(full_path, 'w+', encoding=encoding) as file:
        json.dump(content, file)

def read_json(filename, encoding: str = 'utf-8'):
    with open(filename, 'r+', encoding=encoding) as file:
        value = json.load(file)
    return value

### Write Execute

In [34]:
write_json_file('header_obesity', DICT_HEADER_OBESITY_TRANSLATION)
write_json_file('value_obesity', DICT_VALUE_OBESITY_TRANSLATION)
write_json_file('frequency_obesity', DICT_FREQUENCY_OBESITY)
write_json_file('level_obesity', DICT_LEVEL_OBESITY)


# # Leitura do Json
# value = read_json('filename')

In [35]:
# Etapa 8: Salvar Parquet tratado
output_csv = os.path.join(SILVER_PATH, "obesity_tratado.csv")
df.to_csv(output_csv)

# Etapa 8: Salvar Parquet tratado
output_parquet = os.path.join(SILVER_PATH, "obesity_tratado.parquet")
df.to_parquet(output_parquet)

# Etapa 9 (opcional): também salvar como XLSX
output_xlsx = os.path.join(SILVER_PATH, "obesity_tratado.xlsx")
df.to_excel(output_xlsx, index=False)

# Mensagem final
print("Arquivo tratado salvo com sucesso em:")
print(f"- CSV: {output_parquet}")

Arquivo tratado salvo com sucesso em:
- CSV: /home/lucas-nunes/workspace/Postech/challenges/4_obesity/data/silver/obesity_tratado.parquet


In [36]:
df_test = pd.read_parquet('/home/lucas-nunes/workspace/Postech/challenges/4_obesity/data/silver/obesity_tratado.parquet')
df_test

,gender,age,height,weight,family_history,favc,fcvc,ncp,caec,smoke,ch2o,scc,faf,tue,calc,mtrans,obesity
0,Feminino,21,1.620000,64.000000,True,False,2.0,3.0,1,False,2.000000,False,0.000000,1.000000,0,Transporte público,1
1,Feminino,21,1.520000,56.000000,True,False,3.0,3.0,1,True,3.000000,True,3.000000,0.000000,1,Transporte público,1
2,Masculino,23,1.800000,77.000000,True,False,2.0,3.0,1,False,2.000000,False,2.000000,1.000000,2,Transporte público,1
3,Masculino,27,1.800000,87.000000,False,False,3.0,3.0,1,False,2.000000,False,2.000000,0.000000,2,Caminhada,2
4,Masculino,22,1.780000,89.800000,False,False,2.0,1.0,1,False,2.000000,False,0.000000,0.000000,1,Transporte público,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,Feminino,21,1.710730,131.408528,True,True,3.0,3.0,1,False,1.728139,False,1.676269,0.906247,1,Transporte público,6
2107,Feminino,22,1.748584,133.742943,True,True,3.0,3.0,1,False,2.005130,False,1.341390,0.599270,1,Transporte público,6
2108,Feminino,23,1.752206,133.689352,True,True,3.0,3.0,1,False,2.054193,False,1.414209,0.646288,1,Transporte público,6
2109,Feminino,24,1.739450,133.346641,True,True,3.0,3.0,1,False,2.852339,False,1.139107,0.586035,1,Transporte público,6


In [38]:
df_test['caec'].drop_duplicates()

0     1
10    2
15    3
44    0
Name: caec, dtype: int64